# AI for Psycholinguisitcs
主要内容：使用自然语言处理模型提取语言特征和进行语言任务

## 0. 加载工具包

* 使用到的自然语言处理工具包
    - **srilm**: 计算频次、转移概率。[[官方文档]](https://srilm-python.readthedocs.io/en/latest/#)。安装srilm可参考[[安装教程]](https://github.com/zhaoyanpeng/pysrilm)
    - **hanlp**: 适用于基本自然语言处理任务，包括分词、词性分析、句法分析、语义分析、静态词向量提取等等，对于中文比较友好。[[官方文档]](https://hanlp.hankcs.com/docs/)
    - **Huggingface系列**: 调用开源深度学习模型完成以上两者提到的，以及更多其他的任务，包括情感分析、文本生成等等。[[官网]](https://huggingface.co/)
* 其他常用的自然语言处理工具包
    - **nltk**：可以调用众多语料库（如wordnet等），也可以进行一系列的自然语言处理任务。[[官方文档]](https://www.nltk.org/)
    - **spacy**：速度快、功能全面的自然语言处理工具包。[[官方文档]](https://spacy.io/)
    - **stanza**：Stanford CoreNLP的python版本
    - **fastNLP**：复旦大学制作的NLP工具包

In [195]:
# Install srilm: Refer to https://github.com/zhaoyanpeng/pysrilm

# 如果在colab等服务器上运行，先用以下命令去掉#安装工具包
#!pip install hanlp
#!pip install transformers, tokenizers

In [196]:
# 如果只需要提取一部分特征，可以选择性地导入以下工具包
import os
import re
import json
from collections import Counter, OrderedDict
from tqdm import tqdm

# 数据处理及可视化
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rc("font", family='SimHei') # 用来显示中文，对于macos系统需要换一个支持的字体

# 自然语言处理
from srilm import LM
import hanlp
import torch
import torch.nn.functional as F
from transformers import (
    BertTokenizer,
    GPT2LMHeadModel, 
    TextGenerationPipeline,
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    AutoModelForSeq2SeqLM,
    pipeline
    )


## 1. 词汇及语义特征提取

### 1.1 数据预处理：加载语料库以及进行分词

In [197]:
def filter_str(astr, tokenizer):
    '''
    # 使用分词模型来分词
    输入: 
        astr: str, a sentence
        tokenizer: hanlp tokenizer
    输出:
        a sentence with words separated by space
    '''
    words = tokenizer(astr)
    return ' '.join(words)

def prepare_corpus(tokenizer, corpus, save_json_name):
    '''
    # 对语料库进行分词
    输入:
        tokenizer: hanlp tokenizer
        corpus: str, the path of corpus
        save_json_name: str, the path of saving json file
    输出: 
        
    '''
    with open(save_json_name, 'r', encoding='utf-8') as fp:
        wiki_texts = json.load(fp)
        wiki_texts_new = []
        for line in tqdm(wiki_texts):
            wiki_texts_new.append(filter_str(line, tokenizer))
        open(corpus, 'w').write('\n'.join(wiki_texts_new))

# 加载hanlp中的分词模型
hanlp_tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
# wiki语料
wiki_file = './srilm_data_model/wiki_demo/wiki_z.json'
# 分词后语料文件
wiki_file_tkd = './srilm_data_model/wiki_demo/wiki_z_word.txt'
# 执行
prepare_corpus(hanlp_tok, wiki_file_tkd, wiki_file)

100%|██████████| 1/1 [00:00<00:00,  2.23it/s]


### 1.2 基于语料库统计的N-gram计算

#### 1.2.1 从语料库中生成N-gram模型
* 将语料库（corpus）和指定的模型设置（ngram）输入模型，在模型存储路径（model_path）中输出统计好的模型
* 现成的N-gram语料库：[google n-gram](http://storage.googleapis.com/books/ngrams/books/datasetsv2.html)

In [198]:
def generate_model(model_path, ngram, corpus):
    '''
    输入:
        model_path: str, ngram模型的保存路径
        ngram: str, ngram-count路径
        corpus: str, corpus路径
    输出:
        
    '''
    cmd = '{} -text {} -order 3 -kndiscount3 -lm {}'.format(ngram, corpus, model_path)
    os.system(cmd)

ngram = '/home/zhang/acoustic_theory/workspace/21-12-30-srilm/srilm/bin/i686-m64/ngram-count'
wiki_file_tkd = './srilm_data_model/wiki_demo/wiki_z_word.txt'
model_path = './srilm_data_model/wiki_demo/wiki_z_word.lm'
generate_model(model_path, ngram, wiki_file_tkd)

In [199]:
model_path = './srilm_data_model/wiki/wiki_z_word.lm'
lm = LM(model_path, lower=True) # 加载N-gram模型

#### 1.2.2 采用N-gram模型计算词频
用srilm的LM来调用刚刚生成的模型，采用`lm.logprob_strings(word, context)`来生成 $\log{p \left( \rm{word} | context \right)}$，word是当前单词，当context是空列表`[]`时相当于1-gram即词频

In [200]:
# 计算词频
print('*'*20 + ' 计算词频 ' + '*'*20)
word_freq0_ = lm.logprob_strings('的', [])
word_freq1_ = lm.logprob_strings('西瓜', [])
word_freq2_ = lm.logprob_strings('桌子', [])

# 输出结果
print('='*20 + 'P(的) vs P(西瓜) vs P(桌子)' + '='*20)
print('P(的): ' + str(word_freq0_))
print('P(西瓜): ' + str(word_freq1_))
print('P(桌子): ' + str(word_freq2_))

******************** 计算词频 ********************
====================P(的) vs P(西瓜) vs P(桌子)====================
P(的): -1.3277089595794678
P(西瓜): -5.5793938636779785
P(桌子): -5.5162577629089355


#### 1.2.3 采用N-gram模型计算转移概率

当$n>1$时，在`context`中放入前$n-1$个词，顺序是从右到左。

In [201]:
tp1_ = lm.logprob_strings('西瓜', ['吃', '喜欢'])
tp2_ = lm.logprob_strings('桌子', ['吃', '喜欢'])
print('='*10 + 'P(西瓜 | 吃, 喜欢) vs P(桌子 | 吃, 喜欢)' + '='*10)
print('P(西瓜 | 吃, 喜欢): ' + str(tp1_))
print('P(桌子 | 吃, 喜欢): ' + str(tp2_))

==========P(西瓜 | 吃, 喜欢) vs P(桌子 | 吃, 喜欢)==========
P(西瓜 | 吃, 喜欢): -2.884925365447998
P(桌子 | 吃, 喜欢): -6.211382865905762


#### 1.2.4 采用N-gram模型计算surprisal
$\rm{surprisal} = -\log{ \it{p} \left( \rm{word} | context \right)}$，所以只要取负即可。

In [202]:
s1_ = -lm.logprob_strings('西瓜', ['吃', '喜欢'])
s2_ = -lm.logprob_strings('桌子', ['吃', '喜欢'])
print('='*10 + 'surprisal(西瓜 | 吃, 喜欢) vs surprisal(桌子 | 吃, 喜欢)' + '='*10)
print('surprisal(西瓜 | 吃, 喜欢): ' + str(s1_))
print('surprisal(桌子 | 吃, 喜欢): ' + str(s2_))

==========surprisal(西瓜 | 吃, 喜欢) vs surprisal(桌子 | 吃, 喜欢)==========
surprisal(西瓜 | 吃, 喜欢): 2.884925365447998
surprisal(桌子 | 吃, 喜欢): 6.211382865905762


#### 1.2.5 采用N-gram模型计算entropy
$\rm{entropy} = \sum \left( p*surprisal \right)$，所以对于给定的context，对所有的词来计算surprisal然后求期望

In [203]:
model_path = './srilm_data_model/wiki/wiki_z_morpheme.lm'
lm = LM(model_path, lower=True) # 加载N-gram模型
def entropy_cal(lm, context):
    # entropy
    raw_text_idx = [lm.vocab.intern(w) for w in context]
    vocab_num = lm.vocab.max_interned() + 1
    logprobs = [lm.logprob(i, raw_text_idx) for i in range(vocab_num)]
    logprobs_np = np.array(logprobs)
    logprobs_np_ = logprobs_np[logprobs_np > -np.inf]
    entropy_ = sum(-np.power(10, logprobs_np_)*logprobs_np_)
    return entropy_

print('='*10 + 'entropy(蝴) vs entropy(。)' + '='*10)
e1_ = entropy_cal(lm, ['蝴'])
print('entropy(蝴): ' + str(e1_))
e2_ = entropy_cal(lm, ['。'])
print('entropy(。): ' + str(e2_))

==========entropy(蝴) vs entropy(。)==========
entropy(蝴): 0.03182660213747036
entropy(。): 2.5136258206385347


### 1.3 基于深度学习模型的转移概率计算

以gpt-2为例，采用的模型为[gpt2-chinese-cluecorpussmall](https://huggingface.co/uer/gpt2-chinese-cluecorpussmall)

#### 1.3.1 加载模型，包括分词模型与语言模型


In [204]:
from transformers import BertTokenizer, GPT2LMHeadModel
ckpt_path = "uer/gpt2-chinese-cluecorpussmall" # checkpoint模型路径
tokenizer = BertTokenizer.from_pretrained(ckpt_path) # 分词器
model = GPT2LMHeadModel.from_pretrained(ckpt_path) # 语言模型

#### 1.3.2 获取模型的转移概率

In [205]:
model.config.output_hidden_states = True  # 在模型设置config中设置为True，可以让模型输出hidden states
inputs = tokenizer('蝴蝶飞舞。绵羊吃草。', return_tensors="pt") # 对句子进行分词

tks = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
print('='*10 + '输入tokens: ' + '='*10)
print(tks)

outputs = model(**inputs)  # 将分词后的句子输入模型，得到模型输出的结果
probs = outputs.logits[0]
print('='*10 + '转移概率维度: ' + '='*10)
print(str(probs.shape) + '  输入字数 x 总字数')

==========输入tokens: ==========
['[CLS]', '蝴', '蝶', '飞', '舞', '。', '绵', '羊', '吃', '草', '。', '[SEP]']
==========转移概率维度: ==========
torch.Size([12, 21128])  输入字数 x 总字数


#### 1.3.3 获取模型的surprisal与entropy

In [206]:
probs_sfm = F.softmax(probs, dim=-1)
input_ids_ = inputs['input_ids'][0]
prob_target = [probs_sfm[idx, in_id_tmp].item() for idx, in_id_tmp in enumerate(input_ids_[1:])]

gpt_surprisal = -np.log10(prob_target)
gpt_entropy = -(torch.log10(probs_sfm) * probs_sfm).nansum(dim=-1)
tks = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
for idx in range(len(tks)-1):
    print('\n==========================================')
    print(f'previous tokens: {tks[:idx+1]}')

    prob_sort_idx = probs_sfm[idx, :].argsort(descending=True)
    pred_tks = tokenizer.convert_ids_to_tokens(prob_sort_idx)
    print(f'tokens (top k): {pred_tks[:10]}')
    # print(f'prob of tokens (top k): {probs_sfm[idx, prob_sort_idx[:10]]}')

    print(f'surprisal of {tks[idx+1]}: {gpt_surprisal[idx]}; entropy: {gpt_entropy[idx]}; ')


previous tokens: ['[CLS]']
tokens (top k): ['如', '很', '这', '有', '不', '书', '你', '为', '我', '一']
surprisal of 蝴: 4.849288742842096; entropy: 2.7137670516967773; 

previous tokens: ['[CLS]', '蝴']
tokens (top k): ['蝶', '蜓', '[UNK]', '蛹', '-', '蝴', '蜢', '##ser', '##e', '~']
surprisal of 蝶: 0.00013581902527431; entropy: 0.002098201308399439; 

previous tokens: ['[CLS]', '蝴', '蝶']
tokens (top k): ['蝶', '（', '变', '的', '是', '酥', '属', '超', '结', '飞']
surprisal of 飞: 1.875799630285334; entropy: 2.705556869506836; 

previous tokens: ['[CLS]', '蝴', '蝶', '飞']
tokens (top k): ['机', '行', '蝶', '蛾', '舞', '翔', '碟', '鸟', '鱼', '龙']
surprisal of 舞: 1.3007874730402909; entropy: 1.6138794422149658; 

previous tokens: ['[CLS]', '蝴', '蝶', '飞', '舞']
tokens (top k): ['蝴', '蝶', '的', '是', '（', '(', '，', '-', '[SEP]', '《']
surprisal of 。: 2.868855494287529; entropy: 2.5130739212036133; 

previous tokens: ['[CLS]', '蝴', '蝶', '飞', '舞', '。']
tokens (top k): ['蝴', '[SEP]', '蝶', '这', '一', '是', '。', '我', '不', '飞']
surprisa

### 1.4 词性

In [207]:
## 0. 分词
sent_ex = '这个门被锁了，锁很难被打开。'
tok = hanlp.load(hanlp.pretrained.tok.COARSE_ELECTRA_SMALL_ZH)
tks = tok(sent_ex)
print('0. 分词结果：')
print(tks)

## 1. 词性标注
pos = hanlp.load(hanlp.pretrained.pos.CTB9_POS_ELECTRA_SMALL)
print('1. 词性标注：')
print(pos(tks))

Building model ...          

0. 分词结果：
['这个', '门', '被', '锁', '了', '，', '锁', '很难', '被', '打开', '。']


1. 词性标注：
['DT', 'NN', 'SB', 'VV', 'SP', 'PU', 'VV', 'AD', 'SB', 'VV', 'PU']


### 1.5 词向量

#### 1.5.1 获取静态词向量：以word2vec为例
* hanlp支持调用各种静态词向量， 包括[word2vec](https://hanlp.hankcs.com/docs/api/hanlp/pretrained/word2vec.html), [glove](https://hanlp.hankcs.com/docs/api/hanlp/pretrained/glove.html)等等，具体的模型及文献可以在链接文档中进行选择，一般情况下维度越高越准确。

In [208]:
word2vec = hanlp.load(hanlp.pretrained.word2vec.MERGE_SGNS_BIGRAM_CHAR_300_ZH) # 加载word2vec词向量
word2vec('中国')

tensor([ 1.4234e-02,  8.3600e-02,  2.4145e-02, -1.0256e-01, -1.0829e-01,
        -2.6786e-02, -9.6481e-02,  9.0537e-02, -5.4941e-02,  4.5936e-02,
        -4.2577e-02, -5.1776e-02,  4.9661e-02, -3.2703e-02, -6.6407e-03,
         9.8313e-03,  4.2377e-02, -7.1969e-02,  6.7363e-02, -1.2679e-01,
         1.3423e-03,  1.8129e-02,  1.3923e-02,  6.0298e-02,  2.9974e-02,
         3.4969e-02,  4.7053e-02, -1.4874e-02,  6.6235e-02, -1.5579e-01,
        -1.1716e-01,  8.8726e-02,  6.0976e-02, -8.0692e-02, -3.1017e-02,
        -1.3132e-02,  5.4841e-02,  4.0733e-02, -1.5295e-01, -7.8516e-02,
         6.6119e-02,  2.9393e-02, -3.0162e-02, -4.3704e-02,  8.3047e-03,
        -7.7654e-02, -1.5644e-02,  6.2678e-02,  7.3149e-02, -1.9128e-02,
         2.7543e-02, -1.4893e-02, -1.2223e-02,  9.6474e-02,  2.1985e-02,
         4.4640e-02, -2.4626e-02,  9.8536e-02, -1.3777e-01,  5.1621e-02,
         9.5042e-02, -3.2784e-02,  2.8697e-02, -1.3267e-02,  1.1536e-02,
        -9.0047e-02, -7.2654e-02, -8.7082e-04, -3.6

- 捕获了性别信息
- 捕获了首都信息

In [209]:
print(torch.nn.functional.cosine_similarity(
    word2vec('国王')-word2vec('王妃'), 
    word2vec('男')-word2vec('女'), dim=0)
      )
print(torch.nn.functional.cosine_similarity(
    word2vec('公主')-word2vec('王妃'), 
    word2vec('男')-word2vec('女'), dim=0)
      )

tensor(0.1429, device='cuda:0')
tensor(0.0366, device='cuda:0')


In [210]:
print(torch.nn.functional.cosine_similarity(
    word2vec('日本')-word2vec('东京'), 
    word2vec('中国')-word2vec('北京'), dim=0)
      )
print(torch.nn.functional.cosine_similarity(
    word2vec('韩国')-word2vec('东京'), 
    word2vec('中国')-word2vec('北京'), dim=0)
      )


tensor(0.4674, device='cuda:0')
tensor(0.3933, device='cuda:0')


- 计算相似词

In [211]:
# 单个词
print(word2vec.most_similar('北京')) 
print('\n')

{'上海': 0.6443496942520142, '天津': 0.6384099721908569, '西安': 0.611718475818634, '南京': 0.6113559603691101, '北京市': 0.6093109846115112, '海淀': 0.6049214601516724, '广州': 0.5977935791015625, '京城': 0.5955069661140442, '沈阳': 0.5865166187286377, '深圳': 0.580772876739502}




#### 1.5.2 获取基于上下文的词向量：语言模型的隐藏层表征

同样以1.3中调用的[gpt2-chinese-cluecorpussmall](https://huggingface.co/uer/gpt2-chinese-cluecorpussmall)为例

In [212]:
from transformers import BertTokenizer, GPT2LMHeadModel, TextGenerationPipeline
ckpt_path = "uer/gpt2-chinese-cluecorpussmall" # checkpoint模型路径
tokenizer = BertTokenizer.from_pretrained(ckpt_path) # 分词器
model = GPT2LMHeadModel.from_pretrained(ckpt_path) # 语言模型

In [213]:
model.config.output_hidden_states = True
inputs = tokenizer('小明喜欢吃西瓜。小明喜欢打篮球。小明经常去花店', return_tensors="pt")
outputs = model(**inputs)

print('\n' + '='*10 + '最后一层输出的内隐表征维度: ' + '='*10)
print(str(outputs.hidden_states[-1].shape) + '  1 x 输入字数 x 表征维度')


==========最后一层输出的内隐表征维度: ==========
torch.Size([1, 25, 768])  1 x 输入字数 x 表征维度


## 2. 句法特征提取

### 2.1 句法特征抽取

In [214]:
import hanlp

In [215]:
## workshop中的例子，研究中一般会把标点去掉，但是这里保留了标点，模型也是能够解析标点的
Hanlp = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH) # 选择使用的模型
doc = Hanlp('欢迎大家参加工作坊！', tasks=['dep', 'con']) # 在tasks中选择需要的任务，如果不设置就进行所有任务（运行起来会慢一点）
doc.pretty_print()

#### 2.1.1 成分句法

成分句法输出得到的是一个树结构的数据，可以看作一个嵌套的列表。我们可以：
* 访问句法树的一些属性
* 转换为括号表示法，计算括号数量
* 访问句法树的子树

In [216]:
Hanlp = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH)
doc = Hanlp('欢迎大家参加工作坊！')
tree = doc['con']

In [217]:
# 叶结点的位置
for i in range(len(tree.leaves())):
    print(tree.leaf_treeposition(i))

(0, 0, 0, 0)
(0, 0, 1, 0, 0)
(0, 0, 2, 0, 0, 0)
(0, 0, 2, 0, 1, 0, 0)
(0, 1, 0)


In [218]:
tree[0, 0, 1, 0, 0]

'大家'

In [219]:
# 转为括号表示法
bracket_form = tree.pformat().replace ('\n', '').replace(' ', '') # 去掉换行和空格
bracket_form

'(TOP(IP(VP(VV欢迎)(NP(PN大家))(IP(VP(VV参加)(NP(NN工作坊)))))(PU！)))'

In [220]:
# 转换为Chomsky Normal Form，可以用tree.un_chomsky_normal_form()转换回来
tree.chomsky_normal_form() 
bracket_form = tree.pformat().replace ('\n', '').replace(' ', '')
print(bracket_form)

(TOP(IP(VP(VV欢迎)(VP|<NP-IP>(NP(PN大家))(IP(VP(VV参加)(NP(NN工作坊))))))(PU！)))


In [221]:
# 输出中有些节点只派生出一支，是冗余的（例如最外层的TOP根结点只派生出IP，以及句子中的IP只派生出VP），可以选择压缩节点
tree.collapse_unary(collapseRoot=True, joinChar='|') # 压缩冗余节点，压缩的节点用｜来表示
bracket_form = tree.pformat().replace ('\n', '').replace(' ', '')
bracket_form 

'(TOP|IP(VP(VV欢迎)(VP|<NP-IP>(NP(PN大家))(IP|VP(VV参加)(NP(NN工作坊)))))(PU！))'

In [222]:
import re
import pandas as pd
# 计算括号表示法中每个词的括号数
bracket_clean= re.sub("([^()])", "", bracket_form) # 只保留括号
print(bracket_clean)

# 计算左括号数
left_bracket = [len(re.findall("\(", i)) for i in bracket_clean] 
left_bracket_count = []
for i in left_bracket:
    if len(left_bracket_count) == 0 or (i == 1 and j != 1):
        left_bracket_count.append(1)
    elif i == 1 and j == 1:
        left_bracket_count[-1] += 1
    j = i
print("左括号数:", left_bracket_count)

# 计算右括号数
right_bracket = [len(re.findall("\)", i)) for i in bracket_clean] 
right_bracket_count = []; j = 0
for i in right_bracket:
    if i == 1 and j != 1:
        right_bracket_count.append(1)
    elif i == 1 and j == 1:
        right_bracket_count[-1] += 1
    j = i
print("右括号数:", right_bracket_count)

# 可以保存为 dataframe 进行进一步的句法特征分析
df_bracket = pd.DataFrame([tree.leaves(), left_bracket_count, right_bracket_count]).T
df_bracket.columns = ['word', 'left_bracket', 'right_bracket']
# df_bracket.to_csv('bracket.csv', index=False) # 保存为csv文件
df_bracket

((()((())(()(()))))())
左括号数: [3, 3, 2, 2, 1]
右括号数: [1, 2, 1, 5, 2]


,word,left_bracket,right_bracket
0,欢迎,3,1
1,大家,3,2
2,参加,2,1
3,工作坊,2,5
4,！,1,2


In [223]:
# 句法树的属性
print("Terminal nodes:", tree.leaves())
print("Tree depth:", tree.height())
print("Tree productions:", tree.productions())
print("Part of Speech:", tree.pos())

Terminal nodes: ['欢迎', '大家', '参加', '工作坊', '！']
Tree depth: 7
Tree productions: [TOP|IP -> VP PU, VP -> VV VP|<NP-IP>, VV -> '欢迎', VP|<NP-IP> -> NP IP|VP, NP -> PN, PN -> '大家', IP|VP -> VV NP, VV -> '参加', NP -> NN, NN -> '工作坊', PU -> '！']
Part of Speech: [('欢迎', 'VV'), ('大家', 'PN'), ('参加', 'VV'), ('工作坊', 'NN'), ('！', 'PU')]


In [224]:
# 句法树的嵌套结构
for i in tree.subtrees():  # 根据Tree productions，遍历所有的子树，每一棵子树都是一个Tree对象，可以进行之前相同的操作
    print(i)

(TOP|IP
  (VP
    (VV 欢迎)
    (VP|<NP-IP> (NP (PN 大家)) (IP|VP (VV 参加) (NP (NN 工作坊)))))
  (PU ！))
(VP (VV 欢迎) (VP|<NP-IP> (NP (PN 大家)) (IP|VP (VV 参加) (NP (NN 工作坊)))))
(VV 欢迎)
(VP|<NP-IP> (NP (PN 大家)) (IP|VP (VV 参加) (NP (NN 工作坊))))
(NP (PN 大家))
(PN 大家)
(IP|VP (VV 参加) (NP (NN 工作坊)))
(VV 参加)
(NP (NN 工作坊))
(NN 工作坊)
(PU ！)


In [225]:
# 通过索引访问句法树的子树
treepositions = tree.treepositions() # 所有节点的索引
treepositions

[(),
 (0,),
 (0, 0),
 (0, 0, 0),
 (0, 1),
 (0, 1, 0),
 (0, 1, 0, 0),
 (0, 1, 0, 0, 0),
 (0, 1, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 0, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 1, 0),
 (0, 1, 1, 1, 0, 0),
 (1,),
 (1, 0)]

In [226]:
for i in treepositions: # 遍历所有节点
    print(tree[i])

(TOP|IP
  (VP
    (VV 欢迎)
    (VP|<NP-IP> (NP (PN 大家)) (IP|VP (VV 参加) (NP (NN 工作坊)))))
  (PU ！))
(VP (VV 欢迎) (VP|<NP-IP> (NP (PN 大家)) (IP|VP (VV 参加) (NP (NN 工作坊)))))
(VV 欢迎)
欢迎
(VP|<NP-IP> (NP (PN 大家)) (IP|VP (VV 参加) (NP (NN 工作坊))))
(NP (PN 大家))
(PN 大家)
大家
(IP|VP (VV 参加) (NP (NN 工作坊)))
(VV 参加)
参加
(NP (NN 工作坊))
(NN 工作坊)
工作坊
(PU ！)
！


#### 2.1.2 依存句法
* 依存句法的数据结构更加简单，为一个列表`[(head, relation), ... ]`。列表中第$i$个值中包括了它的核心词的位置以及它与核心词之间的依存关系

In [227]:
Hanlp = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH)
doc = Hanlp('欢迎大家参加工作坊！')
doc['dep']

[(0, 'root'), (1, 'dobj'), (1, 'dep'), (3, 'dobj'), (1, 'punct')]

In [228]:
# 可以保存为 dataframe 进行进一步的句法特征分析
df_dep = pd.DataFrame(doc['dep'], columns=['head', 'rel'])
df_dep['word'] = doc['tok/fine']
df_dep = df_dep[['word', 'head', 'rel']]
df_dep

,word,head,rel
0,欢迎,0,root
1,大家,1,dobj
2,参加,1,dep
3,工作坊,3,dobj
4,！,1,punct


### 2.2 批量操作

只需要将要处理的句子放在list中，一起进行特征抽取即可。这对所有特征都适用，不仅是句法特征。

In [229]:
sentences = ['2023年心理语言学会在广州召开。', '欢迎大家参加工作坊！']
docs = Hanlp(sentences)
docs.pretty_print()

In [230]:
# 提取出来的特征直接索引即可
print("句子数量为:", docs.count_sentences())
for i in range(docs.count_sentences()):
    print(docs['tok/fine'][i])

句子数量为: 2
['2023', '年', '心理', '语言', '学会', '在', '广州', '召开', '。']
['欢迎', '大家', '参加', '工作坊', '！']


# 3.0 语言任务
在本小节中，我们以主题分析任务和上下文学习为例，演示语言模型的加载和推理过程。对于其他语言任务，均可在huggingface平台搜索到类似的教程文档以及代码。
## 3.1 主题分析任务
使用transformers管道pipeline快速实现语言任务

In [231]:
# 从huggingface平台上找到对应的模型路径
model_path = 'model/roberta-base-finetuned-chinanews-chinese'

# 使用transformers工具包加载模型
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 利用pipeline快速进行语言任务
text = '欢迎参加工作坊！'
text_classification = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
res = text_classification(text)[0]
print("="*20, "单个句子主题分析计算", "="*20)
print(f"\nInput: {text}\nPrediction: {res['label']}, Score: {res['score']:.3f}")


# pipeline可以实现批量句子的计算
text_lst = ['2023年心理语言学会在广州召开', '湖人有意签保罗补强，联手詹姆斯追逐总冠军']
res_lst = text_classification(text_lst)
print("\n\n")
print("="*20, "多个句子批量进行主题分析计算", "="*20)
for text, res in zip(text_lst, res_lst):
    print(f"\nInput: {text}\nPrediction: {res['label']}, Score: {res['score']:.3f}")

==================== 单个句子主题分析计算 ====================

Input: 欢迎参加工作坊！
Prediction: culture, Score: 0.723



==================== 多个句子批量进行主题分析计算 ====================

Input: 2023年心理语言学会在广州召开
Prediction: culture, Score: 0.969

Input: 湖人有意签保罗补强，联手詹姆斯追逐总冠军
Prediction: sports, Score: 1.000


## 3.2 上下文学习
通过在上下文中给定任务描述和示例，通用的文本生成模型可以根据上下文快速学习语言任务。在这里我们不使用pipeline，直接调用模型方法进行计算。

In [232]:
# 从huggingface平台上找到对应的模型路径
model_path = "model/flan-t5-large"

# 使用transformers工具包加载模型
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


print("\n\n")
print("="*20, "上下文学习实现文本翻译", "="*20)
text = "translate English to German: How old are you?"

# 调用模型分词器，对输入文本进行分词并转换为模型可处理的tensor形式
input_ids = tokenizer(text, return_tensors="pt").input_ids

# 调用模型的generate方法
outputs = model.generate(input_ids)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens = True)
print(f"Input: {text}\nOutput: {decoded_output}")



print("\n\n")
print("="*20, "上下文学习实现主题文本生成", "="*20)
text = '''Generate sentences with the topic : 
sports => Lionel Messi and MLS club Inter Miami are discussing possible signing
entertainment => 
'''

# 调用模型分词器，对输入文本进行分词并转换为模型可处理的tensor形式
input_ids = tokenizer(text, return_tensors="pt").input_ids

# 调用模型的generate方法
outputs = model.generate(input_ids)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens = True)
print(f"Input: {text}\nOutput: {decoded_output}")





==================== 上下文学习实现文本翻译 ====================


/home/zhang/anaconda3/envs/ngram/lib/python3.7/site-packages/transformers/generation/utils.py:1278: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Input: translate English to German: How old are you?
Output: Wie alte sind Sie?



==================== 上下文学习实现主题文本生成 ====================
Input: Generate sentences with the topic : 
sports => Lionel Messi and MLS club Inter Miami are discussing possible signing
entertainment => 

Output: a new tv series starring adrian sandler is 


## 3.3 文本生成超参数
在本小节中，我们会分析文本生成中的温度参数、搜索策略参数以及top-p参数对文本生成结果的影响。

In [233]:
# 从huggingface平台上找到对应的模型路径
model_path = "model/flan-t5-large"

# 使用transformers工具包加载模型
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

text = 'Welcome to '

# 调用模型分词器，对输入文本进行分词并转换为模型可处理的tensor形式
input_ids = tokenizer(text, return_tensors="pt").input_ids

# 其余可修改参数包括top_k, top_p等, 可直接在.generate()方法中调用
# ref: https://huggingface.co/blog/how-to-generate
print(f'\nInput: {text}\n')
print("="*20, "贪婪搜索", "="*20)
for iter in range(5):
    outputs = model.generate(input_ids, max_length=10)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens = True)
    print(f"Iter {iter}: {decoded_output}")
    
print("="*20, "随机搜索, 温度参数=0.1", "="*20)
for iter in range(5):
    outputs = model.generate(input_ids, do_sample=True, temperature=0.1, max_length=10)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens = True)
    print(f"Iter {iter}: {decoded_output}")
    

print("="*20, "随机搜索, 温度参数=1.0", "="*20)
for iter in range(5):
    outputs = model.generate(input_ids, do_sample=True, temperature=1.0, max_length=10)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens = True)
    print(f"Iter {iter}: {decoded_output}")


Input: Welcome to 

==================== 贪婪搜索 ====================
Iter 0: Welcome to the e-commerce world!
Iter 1: Welcome to the e-commerce world!
Iter 2: Welcome to the e-commerce world!
Iter 3: Welcome to the e-commerce world!
Iter 4: Welcome to the e-commerce world!
==================== 随机搜索, 温度参数=0.1 ====================
Iter 0: Welcome to the iStockphoto
Iter 1: Welcome to the official website of the 
Iter 2: Welcome to the world of e-commerce
Iter 3: Welcome to the world of e-commerce
Iter 4: Welcome to the official website of the 
==================== 随机搜索, 温度参数=1.0 ====================
Iter 0: Welcome to World of Aliens! a
Iter 1: Welcome to the new website.  All
Iter 2: Welcome to the Wikimedia Foundation! This
Iter 3: Hi, I am Jason, the owners of
Iter 4: Welcome to the Official Website of eW
